In [85]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading Dataset**

In [86]:
df_x = pd.read_csv("world_owid.csv", header=0)
df_x.head()

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
0,1750,NaN,9.351,9.351,NaN,NaN,NaN
1,1751,NaN,9.351,9.351,NaN,NaN,NaN
2,1752,NaN,9.354,9.354,NaN,NaN,NaN
3,1753,NaN,9.354,9.354,NaN,NaN,NaN
4,1754,NaN,9.358,9.358,NaN,NaN,NaN


# **Preprocessing Data**

In [87]:

df_x["year"] = pd.to_datetime(df_x["year"], format="%Y")
df_x.index.freq = 'A'

In [88]:
df_x.info()
df = df_x.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         272 non-null    datetime64[ns]
 1   cement_co2   142 non-null    float64       
 2   co2          272 non-null    float64       
 3   coal_co2     272 non-null    float64       
 4   flaring_co2  72 non-null     float64       
 5   gas_co2      140 non-null    float64       
 6   oil_co2      167 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 15.0 KB


In [89]:
test_dates = df_x["year"][130:]
print(test_dates.shape)
#print(df.head())

(142,)


In [90]:
df.tail()

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
267,2017-01-01,1507.923,36096.738,14506.974,391.992,7144.928,12242.628
268,2018-01-01,1569.218,36826.508,14746.831,412.116,7529.847,12266.017
269,2019-01-01,1617.507,37082.559,14725.978,439.254,7647.528,12345.653
270,2020-01-01,1637.537,35264.086,14174.564,407.584,7556.290,11191.809
271,2021-01-01,1672.592,37123.852,14979.598,416.526,7921.830,11837.159


Slicing of Dataframe for co2 and Coal Emissions and data from 1880 onwards

In [91]:
co2_df = df[['year','co2']].loc[(df['year'] >= '1880-01-01')]
coal_df = df[['year','coal_co2']].loc[(df['year'] >= '1880-01-01')]


# **1 Experimenting With Transforms to detrend the data. Log / Exponential Transforms**

In [92]:
# Apply log transformation to 'co2_emission' column
co2_df['Log_Transformed'] = np.log(co2_df['co2'])
coal_df['Log_Transformed'] = np.log(coal_df['coal_co2'])
# Drop NaN rows resulting from the log transformation
co2_df.dropna(inplace=True)
coal_df.dropna(inplace=True)

log_co2_df=co2_df.copy()
log_coal_df=coal_df.copy()

# **2 Differencing**

# After applying Log Transformation, we should differencing for making the data stationary, differencing was applyed 2-3 times depend on data, co2-coal

# log_coal_df

In [93]:
# Apply first-order differencing to the log-transformed values
co2_df['Log_Differenced_1'] = co2_df['Log_Transformed'].diff()
co2_df['Log_Differenced_2'] = co2_df['Log_Differenced_1'].diff()


In [94]:
# Apply first-order differencing to the log-transformed values
coal_df['Log_Differenced_1'] = coal_df['Log_Transformed'].diff()
coal_df['Log_Differenced_2'] = coal_df['Log_Differenced_1'].diff()

In [95]:
log_coal_df =coal_df.dropna().copy()
log_co2_df =co2_df.dropna().copy()

In [96]:
log_coal_df

,year,coal_co2,Log_Transformed,Log_Differenced_1,Log_Differenced_2
132,1882-01-01,912.817,6.816535,0.053619,0.022126
133,1883-01-01,974.149,6.881564,0.065029,0.011410
134,1884-01-01,982.679,6.890283,0.008718,-0.056311
135,1885-01-01,986.996,6.894666,0.004383,-0.004335
136,1886-01-01,995.975,6.903722,0.009056,0.004673
...,...,...,...,...,...
267,2017-01-01,14506.974,9.582385,0.010141,0.034186
268,2018-01-01,14746.831,9.598783,0.016399,0.006258
269,2019-01-01,14725.978,9.597368,-0.001415,-0.017814
270,2020-01-01,14174.564,9.559204,-0.038164,-0.036749


**Augmented Dicky Fuller Test**

In [97]:
from statsmodels.tsa.stattools import adfuller
X = log_coal_df[['year','Log_Differenced_1']].set_index("year").values
log_x = log_coal_df[['year','Log_Differenced_1']].set_index("year").values
result = adfuller(X)
log_result = adfuller(log_x)



print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


print('\n\nADF Statistic: %f' % log_result[0])
print('p-value: %f' % log_result[1])
print('Critical Values:')
for key, value in log_result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.013290
p-value: 0.001342
Critical Values:
	1%: -3.481
	5%: -2.884
	10%: -2.579


ADF Statistic: -4.013290
p-value: 0.001342
Critical Values:
	1%: -3.481
	5%: -2.884
	10%: -2.579


In [98]:
X = log_co2_df[['year','Log_Differenced_2']].set_index("year").values
log_x = log_co2_df[['year','Log_Differenced_2']].set_index("year").values
result = adfuller(X)
log_result = adfuller(log_x)



print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


print('\n\nADF Statistic: %f' % log_result[0])
print('p-value: %f' % log_result[1])
print('Critical Values:')
for key, value in log_result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -7.305057
p-value: 0.000000
Critical Values:
	1%: -3.483
	5%: -2.884
	10%: -2.579


ADF Statistic: -7.305057
p-value: 0.000000
Critical Values:
	1%: -3.483
	5%: -2.884
	10%: -2.579


###### Augmented Dicky fuller test indicates that the data is stationary ADF - after **differencing**

##### Train-test Split

In [99]:
#coal_train_size = int(len(log_coal_df) * 0.9)
#coal_train, coal_test = [0:coal_train_size], X[coal_train_size:len(X)]


co2_data = log_co2_df[['year','Log_Differenced_2']].set_index("year").copy()
co2_train_size = int(len(co2_data) - 10)

co2_train, co2_test = co2_data[0:co2_train_size], co2_data[co2_train_size:len(co2_data)]


In [100]:
co2_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 130 entries, 1882-01-01 to 2011-01-01
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Log_Differenced_2  130 non-null    float64
dtypes: float64(1)
memory usage: 2.0 KB


In [101]:
co2_train = pd.Series(co2_train['Log_Differenced_2'], index=pd.date_range(start='1882-01-01', periods=130, freq='AS-JAN'))
co2_test = pd.Series(co2_test['Log_Differenced_2'], index=pd.date_range(start='2012-01-01', periods=10, freq='AS-JAN'))

**Algorithm 1 - AR - Autoregressive Model**

In [19]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_absolute_error, mean_squared_error

ar_model = AutoReg(co2_train, lags = 7)
ar_model = ar_model.fit()


In [20]:
#print(ar_model.summary())


In [21]:
#forecast_index = pd.date_range(start=f'{2021}-01-01', periods=10, freq='AS-JAN')

forecast = ar_model.predict(start=len(co2_train), end=len(co2_data) - 1, dynamic=False)
forecast

2012-01-01   -0.007360
2013-01-01   -0.002255
2014-01-01    0.007480
2015-01-01   -0.011734
2016-01-01   -0.006454
2017-01-01    0.024441
2018-01-01   -0.010866
2019-01-01   -0.003414
2020-01-01    0.000297
2021-01-01    0.004614
Freq: AS-JAN, dtype: float64

In [22]:
co2_df.tail()

,year,co2,Log_Transformed,Log_Differenced_1,Log_Differenced_2
267,2017-01-01,36096.738,10.493958,0.015989,0.016956
268,2018-01-01,36826.508,10.513973,0.020015,0.004027
269,2019-01-01,37082.559,10.520902,0.006929,-0.013087
270,2020-01-01,35264.086,10.470620,-0.050282,-0.057211
271,2021-01-01,37123.852,10.522015,0.051395,0.101676


In [23]:
co2_test

2012-01-01   -0.018150
2013-01-01   -0.006045
2014-01-01   -0.001612
2015-01-01   -0.007821
2016-01-01   -0.000434
2017-01-01    0.016956
2018-01-01    0.004027
2019-01-01   -0.013087
2020-01-01   -0.057211
2021-01-01    0.101676
Freq: AS-JAN, Name: Log_Differenced_2, dtype: float64

# **Reversing Differencing and Log Transformation on Co2**

In [102]:
# Reverse the first differencing and log transformation for the forecasted values
recovered_test_1 = co2_test.cumsum()
recovered_test_1 = recovered_test_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
recovered_test_2 = recovered_test_1.cumsum()
recovered_test_2 = recovered_test_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
recovered_co2_test = np.exp(recovered_test_2)

# Create a DataFrame to hold the recovered test values
recovered_test_df = pd.DataFrame({
    'Recovered_CO2_test': recovered_co2_test
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(recovered_test_df)

            Recovered_CO2_test
2012-01-01           35006.270
2013-01-01           35319.203
2014-01-01           35577.535
2015-01-01           35558.566
2016-01-01           35524.191
2017-01-01           36096.738
2018-01-01           36826.508
2019-01-01           37082.559
2020-01-01           35264.086
2021-01-01           37123.852


In [25]:
# Reverse the first differencing and log transformation for the forecasted values
recovered_forecast_1 = forecast.cumsum()
recovered_forecast_1 = recovered_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '2011-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
recovered_forecast_2 = recovered_forecast_1.cumsum()
recovered_forecast_2 = recovered_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '2011-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
recovered_co2_forecast = np.exp(recovered_forecast_2)

# Create a DataFrame to hold the recovered forecast values
recovered_forecast_df = pd.DataFrame({
    'Recovered_CO2_Forecast': recovered_co2_forecast
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(recovered_forecast_df)

            Recovered_CO2_Forecast
2012-01-01            35272.763477
2013-01-01            35461.245071
2014-01-01            35918.412964
2015-01-01            35957.081874
2016-01-01            35764.237042
2017-01-01            36452.557598
2018-01-01            36752.607633
2019-01-01            36928.839705
2020-01-01            37116.938860
2021-01-01            37478.522241


In [26]:
# Convert the array to a pandas DataFrame
#df_forecast = pd.DataFrame({'Forecasted_Values': forecast_original_scale})

In [27]:
autoreg_mae = mean_absolute_error(recovered_forecast_df,recovered_test_df)
print(f"MAE : {autoreg_mae:3f}")

MAE : 417.893779


In [28]:
results = pd.DataFrame({'Year': co2_test.index, 'Actual Emission': recovered_test_df['Recovered_CO2_test'], 'Predicted Emission': recovered_forecast_df['Recovered_CO2_Forecast']})

In [29]:
results

,Year,Actual Emission,Predicted Emission
2012-01-01,2012-01-01,35006.270,35272.763477
2013-01-01,2013-01-01,35319.203,35461.245071
2014-01-01,2014-01-01,35577.535,35918.412964
2015-01-01,2015-01-01,35558.566,35957.081874
2016-01-01,2016-01-01,35524.191,35764.237042
2017-01-01,2017-01-01,36096.738,36452.557598
2018-01-01,2018-01-01,36826.508,36752.607633
2019-01-01,2019-01-01,37082.559,36928.839705
2020-01-01,2020-01-01,35264.086,37116.938860
2021-01-01,2021-01-01,37123.852,37478.522241


In [103]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [ ]:
mape_score = mean_absolute_percentage_error(recovered_forecast_df,recovered_test_df)
print(f"MAPE: {mape_score:.2f}%")

**2 ARIMA**

In [64]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np



# Define a list of possible values for p, d, and q
p_values = range(0, 30)  # You can adjust the range based on your requirements
d_values = range(0, 0)  # You can adjust the range based on your requirements
q_values = range(0, 5)  # You can adjust the range based on your requirements

best_mae = np.inf
best_order = None

# Try different combinations of p, d, and q
for p in p_values:
    for d in d_values:
        for q in q_values:
            try:
                # Fit ARIMA model with the current parameters
                model = ARIMA(co2_train, order=(p, d, q))
                model_fit = model.fit()

                # Make predictions on the validation set
                forecast = model_fit.forecast(steps=len(co2_test))

                # Calculate Mean Absolute Error (MAE) as the evaluation metric
                mae = mean_absolute_error(co2_test, forecast)

                # Update the best model if the current model has lower MAE
                if mae < best_mae:
                    best_mae = mae
                    best_order = (p, d, q)

            except Exception as e:
                # Skip the combination if the model fitting fails
                continue

# Fit the best ARIMA model with the selected order parameters
best_model = ARIMA(co2_train, order=(best_order[0], best_order[1], best_order[2]))
best_model_fit = best_model.fit()

# Print the summary of the best ARIMA model
print(best_model_fit.summary())

# Make predictions on the test set using the best ARIMA model
forecast2 = best_model_fit.forecast(steps=10)



# Reverse the first differencing and log transformation for the forecasted values
arima_forecast_1 = forecast2.cumsum()
arima_forecast_1 = arima_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '2011-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
arima_forecast_2 = arima_forecast_1.cumsum()
arima_forecast_2 = arima_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '2011-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
arima_co2_forecast = np.exp(arima_forecast_2)

# Create a DataFrame to hold the recovered forecast values
arima_forecast_df = pd.DataFrame({
    'Recovered_CO2_Forecast': arima_co2_forecast
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(arima_forecast_df)

TypeError: ignored

In [49]:

try:


1

In [ ]:
# ARIMA(14, 4, 6)
#MAE : 2717.503044


#ARIMA(14, 4, 6)
# MAE : 536.429896

ARIMA(17, 3, 9)

In [70]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np



# Define a list of possible values for p, d, and q
p_values = range(10, 20)  # You can adjust the range based on your requirements
d_values = [ 3,4,5]  # You can adjust the range based on your requirements
q_values = range(5, 15)  # You can adjust the range based on your requirements

mae_list = []
best_mae = 10000
best_order = None

# Try different combinations of p, d, and q
for p in p_values:
    for d in d_values:
        for q in q_values:

            try:

                # Fit ARIMA model with the current parameters
                model = ARIMA(co2_train, order=(p, d, q))
                model_fit = model.fit()

                # Make predictions on the validation set
                forecast_ = model_fit.forecast(steps=len(co2_test))

                # Calculate Mean Absolute Error (MAE) as the evaluation metric
                mae = mean_absolute_error(co2_test, forecast_)
                mae_list =mae_list.append(mae)
                # Update the best model if the current model has lower MAE
                if mae < best_mae:
                    best_mae = mae
                    best_order = (p, d, q)

            except Exception as e:
                # Skip the combination if the model fitting fails
                continue

# Fit the best ARIMA model with the selected order parameters
best_model = ARIMA(co2_train, order=(best_order[0], best_order[1], best_order[2]))
best_model_fit = best_model.fit()

# Print the summary of the best ARIMA model
print(best_model_fit.summary())

# Make predictions on the test set using the best ARIMA model
forecast2 = best_model_fit.forecast(steps=10)



# Reverse the first differencing and log transformation for the forecasted values
arima_forecast_1 = forecast2.cumsum()
arima_forecast_1 = arima_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
arima_forecast_2 = arima_forecast_1.cumsum()
arima_forecast_2 = arima_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
arima_co2_forecast = np.exp(arima_forecast_2)

# Create a DataFrame to hold the recovered forecast values
arima_forecast_df = pd.DataFrame({
    'Recovered_CO2_Forecast': arima_co2_forecast
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(arima_forecast_df)

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning

                               SARIMAX Results                                
Dep. Variable:      Log_Differenced_2   No. Observations:                  130
Model:                ARIMA(17, 3, 9)   Log Likelihood                 180.569
Date:                Thu, 10 Aug 2023   AIC                           -307.138
Time:                        16:49:51   BIC                           -230.345
Sample:                    01-01-1882   HQIC                          -275.938
                         - 01-01-2011                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -1.7724      1.317     -1.345      0.178      -4.355       0.810
ar.L2         -1.5722      1.612     -0.975      0.329      -4.732       1.587
ar.L3         -1.3102      1.835     -0.714      0.4

In [119]:
min_mae_index = mae_list.index(min(mae_list))
min_best_order =best_order_[min_mae_index]

In [123]:
min_mae_index

12

In [122]:
min_best_order

(0, 0, 12)

In [125]:
co2_test

2012-01-01   -0.018150
2013-01-01   -0.006045
2014-01-01   -0.001612
2015-01-01   -0.007821
2016-01-01   -0.000434
2017-01-01    0.016956
2018-01-01    0.004027
2019-01-01   -0.013087
2020-01-01   -0.057211
2021-01-01    0.101676
Freq: AS-JAN, Name: Log_Differenced_2, dtype: float64

In [114]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np



# Define a list of possible values for p, d, and q
p_values = range(0, 20)  # You can adjust the range based on your requirements
d_values = [ 0,1,2,3,4,5]  # You can adjust the range based on your requirements
q_values = range(0, 15)  # You can adjust the range based on your requirements

mae_list = []
best_mae = 10000
best_order_ = []

# Try different combinations of p, d, and q
for p in p_values:
    for d in d_values:
        for q in q_values:


          try:
            # Fit ARIMA model with the current parameters
            model = ARIMA(co2_train, order=(p, d, q))
            model_fit = model.fit()

            # Make predictions on the validation set
            forecast_ = model_fit.forecast(steps=len(co2_test))

            # Calculate Mean Absolute Error (MAE) as the evaluation metric
            mae = mean_absolute_error(co2_test, forecast_)
            mae_list.append(mae)
            best_order_.append((p,d,q))
          except Exception as e:
            # Skip the combination if the model fitting fails
            continue

min_mae_index = mae_list.index(min(mae_list))
min_best_order =best_order_[min_mae_index]

# Fit the best ARIMA model with the selected order parameters
best_model = ARIMA(co2_train, order=(min_best_order[0], min_best_order[1], min_best_order[2]))
best_model_fit = best_model.fit()

# Print the summary of the best ARIMA model
print(best_model_fit.summary())

# Make predictions on the test set using the best ARIMA model
forecast2 = best_model_fit.forecast(steps=10)



# Reverse the first differencing and log transformation for the forecasted values
arima_forecast_1 = forecast2.cumsum()
arima_forecast_1 = arima_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
arima_forecast_2 = arima_forecast_1.cumsum()
arima_forecast_2 = arima_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '2010-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
arima_co2_forecast = np.exp(arima_forecast_2)

# Create a DataFrame to hold the recovered forecast values
arima_forecast_df = pd.DataFrame({
    'Recovered_CO2_Forecast': arima_co2_forecast
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(arima_forecast_df)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmo

                               SARIMAX Results                                
Dep. Variable:      Log_Differenced_2   No. Observations:                  130
Model:                ARIMA(0, 0, 12)   Log Likelihood                 210.117
Date:                Thu, 10 Aug 2023   AIC                           -392.234
Time:                        21:21:00   BIC                           -352.088
Sample:                    01-01-1882   HQIC                          -375.921
                         - 01-01-2011                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.001     -0.103      0.918      -0.003       0.003
ma.L1         -0.9395      0.363     -2.590      0.010      -1.650      -0.229
ma.L2         -0.0957      0.111     -0.864      0.3

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [115]:
arima_mae = mean_absolute_error(arima_forecast_df,recovered_test_df)
print(f"MAE : {arima_mae:3f}")

MAE : 2471.952253


In [116]:
mape_score = mean_absolute_percentage_error(arima_forecast_df,recovered_test_df)
print(f"MAPE: {mape_score:.2f}%")

MAPE: 6.21%
